# AutoML: Ray Tune example

First, install the dependencies using `pip` -- which should work well even if you're using `conda` to manage Python packages.
If you've these installed already the commands will do nothing.
In any case, this is needed for using many of the cloud-based services for running notebooks.

In [ ]:
!pip install ray[rllib]
!pip install ray[debug]
!pip install ray[tune]
!pip install bayesian-optimization

In [ ]:
import ray
import ray.tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.bayesopt import BayesOptSearch

In [ ]:
import time

def easy_objective (config, reporter):
    time.sleep(0.2)
    
    for i in range(config["iterations"]):
        reporter(
            timesteps_total=i,
            mean_loss=(config["height"] - 14)**2 - abs(config["width"] - 3)
            )

        time.sleep(0.02)

In [ ]:
ray.init(ignore_reinit_error=True, num_cpus=2)

In [ ]:
space = { "width": (0, 20), "height": (-100, 100) }

algo = BayesOptSearch(
    space,
    metric="mean_loss",
    mode="min",
    utility_kwargs={
        "kind": "ucb",
        "kappa": 2.5,
        "xi": 0.0
    }
)

In [ ]:
# finish quickly for testing
smoke_test = False

config = {
    "num_samples": 10 if smoke_test else 1000,
    "config": {
        "iterations": 10
    },
    "stop": {
        "timesteps_total": 100
    }
}

In [ ]:
scheduler = AsyncHyperBandScheduler(metric="mean_loss", mode="min")

In [ ]:
ray.tune.run(
    easy_objective,
    name="my_exp",
    search_alg=algo,
    scheduler=scheduler,
    **config
)

In [ ]:
ray.shutdown()